In [117]:
import pandas as pd
from scipy.spatial.distance import euclidean
from process import *
from utils import *
from model import *

In [118]:
#prendo i dati relativi al dataset
device='cpu'
layer=3
adj, adj_i, features, labels, idx_train, idx_val, idx_test, num_features, num_labels, g = full_load_data_custom('wisconsin','splits/wisconsin_split_0.6_0.2_0.npz')
features = features.to(device)

# processo la matrice di adiacenza del grafo per darlo in input alla rete
adj = adj.to(device)
adj_i = adj_i.to(device)
list_mat = []
list_mat.append(features)
no_loop_mat = features
loop_mat = features

for ii in range(layer):
    no_loop_mat = torch.spmm(adj, no_loop_mat)
    loop_mat = torch.spmm(adj_i, loop_mat)
    list_mat.append(no_loop_mat)
    list_mat.append(loop_mat)

In [119]:
layer_norm=bool(1)
# Define the path to the saved model checkpoint
PATH = "trained_epochs\wisconsin5.pt"
GNNwisconsin=FSGNN(nfeat=num_features,
                nlayers=len(list_mat),
                nhidden=64,
                nclass=num_labels,
                dropout=0.5).to(device)
GNNwisconsin.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [120]:
#prendo l'embedding dalla rete
GNNwisconsin.eval()
out = GNNwisconsin.emb(list_mat, layer_norm)
#torch.save(out,'wisconsin/emb_wisconsin25.pt')


In [50]:
out.shape

torch.Size([251, 448])

In [121]:
#trasformo la matrice di adiacenza in grafo
G = nx.Graph(g)
adj_mat=pd.DataFrame(columns=['target','source'])
adj_mat = nx.to_pandas_edgelist(G)
adj_mat = adj_mat[adj_mat['source']!=adj_mat['target']]
adj_mat

,source,target,weight
0,0,20,1
1,0,28,1
2,0,99,1
3,0,104,1
4,0,200,1
...,...,...,...
460,209,232,1
461,210,225,1
462,215,242,1
464,233,242,1


In [53]:
'''#aggiungo i pesi ottenuti dall'embedding
df_emb = pd.DataFrame(out.detach().numpy(), columns=['feature_' + str(x) for x in range(448)])
adj_mat['weight'] = adj_mat.apply(lambda x:
                                    manhattan_distance(df_emb.loc[x['target']].values,
                                    df_emb.loc[x['source']].values),
                                    axis=1)
adj_mat.to_csv('wisconsin\wisconsin_cosine_weights1.csv', index = False)'''

"#aggiungo i pesi ottenuti dall'embedding\ndf_emb = pd.DataFrame(out.detach().numpy(), columns=['feature_' + str(x) for x in range(448)])\nadj_mat['weight'] = adj_mat.apply(lambda x:\n                                    manhattan_distance(df_emb.loc[x['target']].values,\n                                    df_emb.loc[x['source']].values),\n                                    axis=1)\nadj_mat.to_csv('wisconsin\\wisconsin_cosine_weights1.csv', index = False)"

448

In [55]:
#aggiungo i pesi ottenuti dall'embedding
df_emb = pd.DataFrame(out.detach().numpy(), columns=['feature_' + str(x) for x in range(448)])
adj_mat['weight'] = adj_mat.apply(lambda x:
                                    cosine_similarity(np.array([df_emb.iloc[x['target']]]),
                                    np.array([df_emb.iloc[x['source']]]))[0][0],
                                    axis=1)
adj_mat.to_csv('weight_epochs\wisconsin_cosine_weights5.csv', index = False)

In [56]:
#label della rete 
pd.DataFrame(labels.numpy(),columns=['label']).to_csv('weight_epochs\wisconsin_labels.csv', index = False)